### 필요 라이브러리 설치 및 가져오기

In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
import konlpy
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer #CountVectorizer # tf-idf 방식을 사용하려면 대신 TfidfVectorizer를 import
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 네이버 현재 상영작 리뷰 크롤링 (이건 재실행하지 말 것)

In [ ]:
# 네이버 평점, 리뷰 페이지
review_data = []

for page in range(1, 1000):
  url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'

  movie = requests.get(url)
  soup = BeautifulSoup(movie.content, 'html.parser')
  score_results = soup.find_all('td', {'class': 'title'})
  
  for review in score_results:
    sentence = review.find('a', {'class': 'report'}).get('onclick').split("', '")[2]
    
    if sentence != "":
      mtitle = review.find('a', {'class': 'movie color_b'}).get_text()
      score = review.find('em').get_text()
      review_data.append([mtitle, sentence, int(score)])
      # review_cnt -= 1

In [ ]:
columns_name = ['movie', 'review', 'score']
with open('naver_review.csv', 'w', newline='', encoding='utf8') as f:
  write = csv.writer(f)
  write.writerow(columns_name)
  write.writerows(review_data)

### 데이터프레임 변환 및 오류 여부 확인

In [ ]:
df = pd.read_csv('naver_review.csv')
df

,movie,review,score
0,관계의 일변,내용이라도 있으면 좋겠구만~~이야기 하고픈 내용이 뭔지 궁금,1
1,비상선언,감독 어쩌냐... 이제 비행기탈때마다 ㅂㄷㅂㄷ 할거같은데,1
2,더 퍼스트 슬램덩크,두번봐 세번봐 네번보세요,10
3,자백,영화제목이 미쓰네요 이미 반전의 결과를 알려 주고 있습니다. 원작을 먼저 보는것을 ...,4
4,자백,너무좋아요재미있어요,10
...,...,...,...
9321,동감,잔잔하며 옛 과거를 생각하게 됨,8
9322,유령,배우분들 연기 좋은 건 인정 스토리는 개인적으로 아쉽,7
9323,정이,감독작품은 죄다 어디서 본듯한.이영화 저영화 짜깁기한 수준에 오버스런 캐릭터 진심...,1
9324,애드 아스트라,결국 없다는 것. 그것을 받아들이는 것. 삶에서 중요한 많은 것들.,10


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9326 entries, 0 to 9325
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   movie   9326 non-null   object
 1   review  9326 non-null   object
 2   score   9326 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 218.7+ KB


In [ ]:
# comment에 내용이 없는 행 파악
print(len(df[df['review'] == '']))

# comment에 내용이 없는 행은 지워서 새 df에 저장
reviews_with_comment_df = df[df['review'] != '']
reviews_with_comment_df.reset_index(drop=True, inplace=True)

print(len(reviews_with_comment_df))  # comment가 있는 리뷰의 수 

0
9326


In [ ]:
# 텍스트를 tokenize해서 adjective, verb, noun만 추출하는 함수

def tokenize_korean_text(text): 
  text_filtered = re.sub('[^,.?!\w\s]','', text)

  okt = konlpy.tag.Okt() 
  Okt_morphs = okt.pos(text_filtered) 

  words = []
  for word, pos in Okt_morphs:
    if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':
      words.append(word)

  words_str = ' '.join(words)
  return words_str

In [ ]:
X_texts = []
y = []

for score, review in zip(reviews_with_comment_df['score'], reviews_with_comment_df['review']):
  if 4 <= score <= 7: 
    continue  
     # 평점이 4~7인 영화는 애매하기 때문에 학습데이터로 사용하지 않음

  tokenized_comment = tokenize_korean_text(review)  # 위에서 만들었던 함수로 comment 쪼개기
  X_texts.append(tokenized_comment)

  y.append(1 if score > 7 else -1)
    # 평점이 8 이상이면(8,9,10) 값을 1로 지정 (positive)
    # 평점이 3 이하이면(1,2,3) 값을 -1로 지정 (negative)

print(f'원래 text 수: {len(reviews_with_comment_df)}')
print(f'평점 3 이하 혹은 8 이상인 text 수: {len(X_texts)}')    
print(X_texts[:5])

원래 text 수: 9326
평점 3 이하 혹은 8 이상인 text 수: 7744
['내용 있으면 좋겠구만 이야기 하고픈 내용 궁금', '감독 어쩌 이제 비행기 탈때 할거 같은데', '번 봐 세번 봐 번 보세요', '좋아요 재미있어요', '좋아요 재미있어요']


### LDA 선형판별분석과 시각화

In [ ]:
reviews_with_comment_df

,movie,review,score
0,관계의 일변,내용이라도 있으면 좋겠구만~~이야기 하고픈 내용이 뭔지 궁금,1
1,비상선언,감독 어쩌냐... 이제 비행기탈때마다 ㅂㄷㅂㄷ 할거같은데,1
2,더 퍼스트 슬램덩크,두번봐 세번봐 네번보세요,10
3,자백,영화제목이 미쓰네요 이미 반전의 결과를 알려 주고 있습니다. 원작을 먼저 보는것을 ...,4
4,자백,너무좋아요재미있어요,10
...,...,...,...
9321,동감,잔잔하며 옛 과거를 생각하게 됨,8
9322,유령,배우분들 연기 좋은 건 인정 스토리는 개인적으로 아쉽,7
9323,정이,감독작품은 죄다 어디서 본듯한.이영화 저영화 짜깁기한 수준에 오버스런 캐릭터 진심...,1
9324,애드 아스트라,결국 없다는 것. 그것을 받아들이는 것. 삶에서 중요한 많은 것들.,10


In [ ]:
import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
# 트위터 형태소 분석기(Okt)를 활용
from konlpy.utils import pprint
from konlpy.tag import Okt
okt = Okt()
print(okt)

In [ ]:
review_data = reviews_with_comment_df['review']
print(review_data[0])

내용이라도 있으면 좋겠구만~~이야기 하고픈 내용이 뭔지 궁금


In [ ]:
# 세 글자 이상의 명사를 사용 (두 글자 이하의 단어는 제거)
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 2:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[0])

100%|██████████| 9326/9326 [00:37<00:00, 246.64it/s]

9326
이야기


In [ ]:
# 사이킷런 패키지 활용 
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[0])

(9326, 2944)
  (0, 1982)	1.0


In [ ]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(n_components=2)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda.fit(review_tfid)

LatentDirichletAllocation(n_components=2)

In [ ]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력 (앞에서 5개의 단어만 출력)
print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 5 })

2944
{'이야기': 1982, '비행기': 1118, '폴버호벤': 2714, '마지막': 730, '인내력': 2032}


In [ ]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items() } 

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{1982: '이야기', 1118: '비행기', 2714: '폴버호벤', 730: '마지막', 2032: '인내력'}


In [ ]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 2157개의 단어)
print(lda.components_.shape)

(2, 2944)


In [ ]:
# 2157개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('스토리', 326.441), ('드라마', 147.294), ('주인공', 84.406), ('영화관', 71.622), ('긴장감', 55.134), ('아바타', 54.657), ('캐릭터', 44.531), ('무조건', 37.592), ('뮤지컬', 37.495), ('황정민', 35.475)]
토픽 유형 2: [('마지막', 158.343), ('이야기', 124.343), ('슬램덩크', 114.709), ('연기력', 62.932), ('영화로', 53.891), ('송태섭', 45.931), ('분위기', 38.464), ('할리우드', 38.183), ('쓰레기', 35.198), ('포인트', 34.204)]


In [ ]:
# pyLDAvis 설치
!pip3 install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached funcy-1.18-py2.py3-none-any.whl (33 kB)
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=c2285de7b5357798c442b151508ec7451cb6c890bfd475f1e5d7dbd925057fab
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=045c0cf8bac859d84d0ef4e4a5abfa61a864148d3387d9cbd24b798e9af84731
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Succ

In [ ]:
# LDA 토픽 모델링 결과를 시각화
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


### 토픽모델링  분석결과

네이버 영화 현재 상영작에 대한 리뷰를 분석하기 위해 토픽 모델링을 수행하였다. 크롤링을 통해 만 개의 데이터를 수집하였고 수집시 결측값은 제외하도록 설계하였다. 그래프를 보면 두 가지 주성분으로 구분이 가능한데, 1번에서 관람객들은 슬램덩크나 아바타, 주인공이 누구인지, 어떤 캐릭터가 있는지 등 영화 자체에 중점을 두었다면 2번에서 관람객은 영화의 작품성을 중요시하는 것으로 해석할 수 있다.

## 로지스틱 분류

In [ ]:
# 사이킷런 패키지 활용
from sklearn.linear_model import LogisticRegression

# 로지스틱 분류 모델링 객체를 생성 
lr = LogisticRegression()

data_for_logit = reviews_with_comment_df
tmp = []
for i in range(len(data_for_logit)):
  a = data_for_logit["score"][i]
  if a > 5:
    tmp.append(1)
  else:
    tmp.append(0)
tmp = pd.DataFrame(tmp)
data_for_logit["labels"] = tmp
labels = data_for_logit["labels"]
# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, labels)

LogisticRegression()

In [ ]:
# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

[1]


### TF-IDF 벡터화

In [ ]:
# train_test_split
X_train_texts, X_test_texts, y_train, y_test = train_test_split(X_texts, y, test_size=0.2, random_state=0)

In [ ]:
# CountVectorizer로 vector화
tf_vectorizer = TfidfVectorizer()
X_train_tf = tf_vectorizer.fit_transform(X_train_texts)  # training data에 맞게 fit & training data를 transform
X_test_tf = tf_vectorizer.transform(X_test_texts) # test data를 transform

vocablist = [word for word, number in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]  # 단어들을 번호 기준 내림차순으로 저장

In [ ]:
print(X_train_tf[:1], '\n')
print(X_test_tf[:1], '\n')
print(vocablist[:3])

  (0, 436)	0.11132466368460438
  (0, 12267)	0.16035601851152642
  (0, 7031)	0.182543841839042
  (0, 13521)	0.11351865741325783
  (0, 8410)	0.182543841839042
  (0, 12805)	0.0733393725242713
  (0, 5060)	0.17435497641584288
  (0, 4987)	0.13932257949323726
  (0, 1894)	0.16403822608618057
  (0, 10803)	0.12592653881367805
  (0, 6060)	0.3370897678694511
  (0, 5061)	0.182543841839042
  (0, 14767)	0.1077915064332961
  (0, 13734)	0.182543841839042
  (0, 11831)	0.13932257949323726
  (0, 11928)	0.09342196515650017
  (0, 10022)	0.182543841839042
  (0, 6765)	0.15454592603040912
  (0, 7945)	0.12718921414453274
  (0, 7493)	0.182543841839042
  (0, 13277)	0.16035601851152642
  (0, 13541)	0.182543841839042
  (0, 979)	0.17435497641584288
  (0, 10276)	0.21659287682317355
  (0, 15011)	0.182543841839042
  (0, 4114)	0.365087683678084
  (0, 4237)	0.365087683678084
  (0, 12278)	0.14635706060721 

  (0, 14686)	0.07343566442516633
  (0, 14450)	0.08748330179416677
  (0, 14399)	0.1893778386168791
  (0, 14340)	0.121

### 로지스틱 회귀 모델 학습

In [ ]:
model = LogisticRegression(C=0.1, penalty='l2', random_state=0)
model.fit(X_train_tf, y_train)  # 학습

LogisticRegression(C=0.1, random_state=0)

In [ ]:
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

LogisticRegression(C=0.1, random_state=0)

In [ ]:
y_test_pred = model.predict(X_test_tf)

print('Misclassified samples: {} out of {}'.format((y_test_pred != y_test).sum(), len(y_test)))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_test_pred)))  # model.score(X_test_tf, y_test)로 계산해도 됨

Misclassified samples: 237 out of 1549
Accuracy: 0.85


In [ ]:
coefficients = model.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:10]:
  print('{0:}({1:.3f})'.format(vocablist[word_num], coef))

print('\n부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-10:]: 
  print('{0:}({1:.3f})'.format(vocablist[word_num], coef))

긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)
최고(0.969)
감동(0.925)
사랑(0.688)
재밌게(0.645)
봤습니다(0.577)
인생(0.534)
봤어요(0.500)
눈물(0.463)
좋아요(0.443)
명작(0.440)

부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)
블랙(-0.533)
아깝다(-0.568)
아까(-0.589)
별로(-0.627)
없음(-0.627)
시간(-0.809)
쓰레기(-0.872)
노잼(-0.940)
마블(-1.049)
최악(-1.214)


### 새로운 댓글의 긍정/부정 예측

In [ ]:
# 긍정/부정 테스트용 함수 생성
def guess_good_or_bad(text):
  text_filtered = text.replace('.', '').replace(',','').replace("'","").replace('·', ' ').replace('=','') 

  okt = konlpy.tag.Okt() 
  Okt_morphs = okt.pos(text_filtered) 

  words = []
  for word, pos in Okt_morphs:
    if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':
      words.append(word)
  words_str = ' '.join(words)
  
  new_text_tf = tf_vectorizer.transform([words_str])

  if model.predict(new_text_tf) == 1:
    print('긍정')
  else:
    print('부정')

In [ ]:
guess_good_or_bad('근래 본 영화 중에 제일 괜찮은듯')

긍정


In [ ]:
guess_good_or_bad('내 돈... 내 눈... 진짜..')